In [1]:
import subprocess
import time
import datetime
import pandas as pd
from pathlib import Path

def get_wifis():
    subprocess.run(
        ['netsh', 'interface', 'set', 'interface', 'name="Wi-Fi"', 'admin=disabled'],
        capture_output=True)
    subprocess.run(['netsh', 'interface', 'set', 'interface', 'name="Wi-Fi"', 'admin=enabled'],
        capture_output=True)

    time.sleep(3)
    output = subprocess.run(
        ['netsh', 'wlan', 'show', 'network', 'mode=Bssid'],
        capture_output=True, text=True, encoding='ISO-8859-1').stdout

    results = output.split('\n\n')[1:-1]

    timestamp = datetime.datetime.now()
    wifis = []
    for result in results:
        lines = result.split('\n')

        for i in range(len(lines)):
            if lines[i].split()[0] == 'BSSID':
                bssid = lines[i].split()[-1]
                rssi = int(lines[i+1].split()[-1][:-1])
                if rssi > 50:
                    wifis.append({'bssid':bssid, 'rssi':rssi, 'timestamp':timestamp})

    return wifis

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

In [3]:
position = input('enter position : ')
model_path = Path('../ml_model') / position

model_rdf = joblib.load(model_path / 'model_rdf.plk')
model_svm = joblib.load(model_path / 'model_svm.plk')

In [4]:
bssids = np.load(model_path / 'features.npy')
bssids

array(['88:36:6c:1d:d7:6e', '88:36:6c:1d:d7:6c', '88:36:6c:1d:ae:8c',
       '88:36:6c:1d:db:e0', '88:36:6c:1d:a7:e4', '88:36:6c:1d:a5:14',
       '88:36:6c:1d:c2:d0', '88:36:6c:1d:ae:8e', '88:36:6c:1d:a5:16',
       '88:36:6c:1d:d8:6c', '88:36:6c:1d:d0:fc', '88:36:6c:1d:b0:d8',
       '88:36:6c:05:6a:e4', '88:36:6c:1d:a5:bc', '88:36:6c:1d:c6:f4',
       '88:36:6c:1d:ab:f8', '88:36:6c:1d:c5:88', '88:36:6c:1d:9f:30',
       '88:36:6c:1d:a1:18', '88:36:6c:1d:a6:20', '88:36:6c:1d:d8:6e',
       '88:36:6c:1d:d6:8c', '88:36:6c:1d:d6:0c', '88:36:6c:1d:dc:0c',
       '88:36:6c:1d:cf:10', '88:36:6c:1d:a5:be', '88:36:6c:1d:c0:6c',
       '90:9f:33:b2:18:b0', '88:36:6c:1d:9e:40', '88:36:6c:1d:d9:10',
       '88:36:6c:1d:d7:bc', '88:36:6c:1d:d7:4c', '88:36:6c:1d:d7:94',
       '70:5d:cc:d0:e8:ee', '0a:30:0d:d2:99:a6', '06:30:0d:d2:99:96',
       '0a:30:0d:d2:99:a1', '06:30:0d:d2:99:91', '00:30:0d:d2:99:a0',
       '00:30:0d:d2:99:90', '88:36:6c:1d:cb:3c', '88:36:6c:1d:c8:48',
       '88:36:6c:1d:

In [6]:
rp_encoder = LabelEncoder()
rp_encoder.classes_ = np.load(model_path / 'classes.npy', allow_pickle=True)
rp_encoder.classes_

array(['751D문쪽', '751D창쪽', '751세면대', '751앞계단', '751앞복도', '7층엘베', '7층휴게실L',
       '7층휴게실R'], dtype=object)

In [15]:
user_wifis = get_wifis()

wifi_dict = defaultdict.fromkeys(bssids, 0)
for user_wifi in user_wifis:
    if user_wifi['bssid'] in wifi_dict:
        wifi_dict[user_wifi['bssid']] = user_wifi['rssi']
        
wifi_dict

defaultdict(None,
            {'88:36:6c:1d:d7:6e': 78,
             '88:36:6c:1d:d7:6c': 0,
             '88:36:6c:1d:ae:8c': 0,
             '88:36:6c:1d:db:e0': 0,
             '88:36:6c:1d:a7:e4': 0,
             '88:36:6c:1d:a5:14': 0,
             '88:36:6c:1d:c2:d0': 0,
             '88:36:6c:1d:ae:8e': 0,
             '88:36:6c:1d:a5:16': 0,
             '88:36:6c:1d:d8:6c': 0,
             '88:36:6c:1d:d0:fc': 0,
             '88:36:6c:1d:b0:d8': 0,
             '88:36:6c:05:6a:e4': 0,
             '88:36:6c:1d:a5:bc': 0,
             '88:36:6c:1d:c6:f4': 0,
             '88:36:6c:1d:ab:f8': 0,
             '88:36:6c:1d:c5:88': 0,
             '88:36:6c:1d:9f:30': 0,
             '88:36:6c:1d:a1:18': 0,
             '88:36:6c:1d:a6:20': 0,
             '88:36:6c:1d:d8:6e': 0,
             '88:36:6c:1d:d6:8c': 0,
             '88:36:6c:1d:d6:0c': 0,
             '88:36:6c:1d:dc:0c': 0,
             '88:36:6c:1d:cf:10': 0,
             '88:36:6c:1d:a5:be': 0,
             '88:36

In [25]:
prediction = model_rdf.predict(pd.DataFrame.from_dict(dict(wifi_dict)))
prediction_rp = rp_encoder.inverse_transform(prediction)[0]

print('current location : '+prediction_rp)

ValueError: If using all scalar values, you must pass an index